<a href="https://colab.research.google.com/github/MaguireMaName/Coursera_Capstone/blob/master/The_Battle_of_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone: Battle of the Neighbourhoods

##Introduction/Business Problem

## Data description

In [0]:
# !pip install geocoder

In [0]:
# load dependancies

import pandas as pd
import numpy as np
import geocoder

In [6]:
nghbrhd_data = pd.read_csv('Canberra suburbs.csv')
print(nghbrhd_data.shape)
nghbrhd_data.head()

(154, 4)


,Suburb,Postcode,Country,Region
0,Barton,2600,Australia,Australian Capital Territory
1,Canberra,2600,Australia,Australian Capital Territory
2,Page,2614,Australia,Australian Capital Territory
3,City,2601,Australia,Australian Capital Territory
4,Canberra,2601,Australia,Australian Capital Territory


In [0]:
Lat_list=[]
Lng_list=[]

for i in range(nghbrhd_data.shape[0]):
    address='{}, Canberra, Australia'.format(nghbrhd_data.at[i,'Postcode'])
    g = geocoder.arcgis(address)
    Lat_list.append(g.latlng[0])
    Lat_list.append(g.latlng[1])


In [0]:
test = pd.DataFrame({'Latitude': Lng_list,
                     'Longitude': Lat_list})